# Working with the new Yr weather API

In [134]:
# some useful imports
import datetime, pytz
import requests
import json
import xmltodict
import xml.etree.ElementTree as ET
import emoji

In [135]:
lat = 53.86893
lon = 10.68729
alt = 162



In [136]:
class API_LocationForecast(object):

    """Class to use the API of api.met.no"""

    base_url = 'https://api.met.no/weatherapi/locationforecast/2.0/classic?'
    forecast_link = 'locationforecast'

    def __init__(self, lat, lon, alt):
        """
        :param double lat: latitude coordinate
        :param double lon: longitude coordinate
        """
        self.coordinates = dict(lat=lat, lon=lon, alt=alt)
        self.url = self.get_url()

    def get_url(self):
        """Return the url of API service"""
        return '{base_url}{location_name}'.format(
            base_url=self.base_url,
            location_name=self.location_name,
        )

In [137]:
# Create locationforecast url for API request

base_url = 'https://api.met.no/weatherapi/locationforecast/2.0/complete?'
api_url = '{base_url}lat={lat}&lon={lon}&altitude={alt}'.format(
    base_url=base_url,
    lat=lat,
    lon=lon,
    alt=alt
)

print(api_url)

https://api.met.no/weatherapi/locationforecast/2.0/complete?lat=53.86893&lon=10.68729&altitude=162


In [138]:
# Send request to API

sitename = 'https://jkl453.github.io/garden-123/'
headers = {'User-Agent': sitename}

response = requests.get(api_url, headers=headers)


print(response.status_code)
                                                                                                                                                                                                                                                             


200


In [139]:
#print(response.text)

response_json = json.loads(response.text)
print(json.dumps(response_json, indent=4))


{
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [
            10.6873,
            53.8689,
            162
        ]
    },
    "properties": {
        "meta": {
            "updated_at": "2022-05-17T12:35:10Z",
            "units": {
                "air_pressure_at_sea_level": "hPa",
                "air_temperature": "celsius",
                "air_temperature_max": "celsius",
                "air_temperature_min": "celsius",
                "air_temperature_percentile_10": "celsius",
                "air_temperature_percentile_90": "celsius",
                "cloud_area_fraction": "%",
                "cloud_area_fraction_high": "%",
                "cloud_area_fraction_low": "%",
                "cloud_area_fraction_medium": "%",
                "dew_point_temperature": "celsius",
                "fog_area_fraction": "%",
                "precipitation_amount": "mm",
                "precipitation_amount_max": "mm",
                "prec

**How to extract Data from json response**

In [146]:
json_snippet = response_json['properties']['timeseries']
print(json.dumps(json_snippet, indent=4))

[
    {
        "time": "2022-05-17T13:00:00Z",
        "data": {
            "instant": {
                "details": {
                    "air_pressure_at_sea_level": 1021.0,
                    "air_temperature": 16.8,
                    "air_temperature_percentile_10": 16.2,
                    "air_temperature_percentile_90": 17.9,
                    "cloud_area_fraction": 81.0,
                    "cloud_area_fraction_high": 35.4,
                    "cloud_area_fraction_low": 30.3,
                    "cloud_area_fraction_medium": 75.4,
                    "dew_point_temperature": 10.6,
                    "fog_area_fraction": 0.0,
                    "relative_humidity": 66.6,
                    "ultraviolet_index_clear_sky": 4.0,
                    "wind_from_direction": 20.8,
                    "wind_speed": 2.9,
                    "wind_speed_of_gust": 6.9,
                    "wind_speed_percentile_10": 1.7,
                    "wind_speed_percentile_90": 3.5
        

In [158]:
date_today = datetime.datetime.now(pytz.timezone("Europe/Berlin"))

print(date_today)

print(date_today.strftime("%c"))

date_today_day = date_today.strftime("%d")
date_today_month = date_today.strftime("%m")
date_today_time = date_today.strftime("%H:%M")

print("")
print(date_today_day)
print(date_today_month)
print(date_today_time)
print("")

todays_date = datetime.datetime.now(pytz.timezone("Europe/Berlin"))
tomorrows_date = todays_date + datetime.timedelta(days=1)

print(todays_date)
print(tomorrows_date)

json_snippet = response_json['properties']['timeseries'][0]['time']
print(type(json_snippet))

print(datetime.datetime.strptime(json_snippet, '%Y-%m-%dT%H:%M:%SZ'))
print(json.dumps(json_snippet, indent=4))


# get time from entries
def get_times(json_snippet):
    times = []
    for entry in json_snippet:
        times.append(datetime.datetime.strptime(entry['time'], '%Y-%m-%dT%H:%M:%SZ'))
    return times

2022-05-17 15:58:18.146041+02:00
Tue May 17 15:58:18 2022

17
05
15:58

2022-05-17 15:58:18.146300+02:00
2022-05-18 15:58:18.146300+02:00
<class 'str'>
2022-05-17 13:00:00
"2022-05-17T13:00:00Z"


# Funktionalität für Wettervorhersage

- [] Wetterbericht heute
- [] Wetterbericht morgen
- [] Wetterbericht für den nächsten persönlichen Gießtag

In der user-Datenbank kann dann immer auch der nächste Gießtag gespeichert werden. Dieser kann dann ausgelesen werden um den entsprechenden Wetterbericht zu generieren.

# Aufbau der Wettervorhersage

**individuelle Anfrage**
- aktueller Zeitpunkt + next 6 hours

**morgentliche Benachrichtigung**
- 06:00 Uhr bis 12:00 Uhr
- 12:00 Uhr bis 18:00 Uhr
- 18:00 Uhr bis 00:00 Uhr

In [142]:
"""individual forecast request"""

# get current date and time
date_today = datetime.datetime.now(pytz.timezone("Europe/Berlin"))
time_now = date_today.strftime("%H:%M")

print(time_now)
print(type(time_now))
print(date_today)

json_snippet = response_json['properties']['timeseries'][0]['data']['next_6_hours']
print(json.dumps(json_snippet, indent=4))

yr_symbol_code = json_snippet['summary']['symbol_code']
yr_precipitation_amount = json_snippet['details']['precipitation_amount']
yr_precipitation_probability = json_snippet['details']['probability_of_precipitation']


15:24
<class 'str'>
2022-05-17 15:24:13.945721+02:00
{
    "summary": {
        "symbol_code": "partlycloudy_day"
    },
    "details": {
        "air_temperature_max": 18.5,
        "air_temperature_min": 15.9,
        "precipitation_amount": 0.0,
        "precipitation_amount_max": 0.0,
        "precipitation_amount_min": 0.0,
        "probability_of_precipitation": 0.2
    }
}


In [143]:
# uni code weather symbols
# https://unicode.org/emoji/charts/full-emoji-list.html
weather_symbols_uni = {
    'sun': '\U00002600',
    'cloud': '\U00002601',
    'sun behind cloud': '\U000026C5',
    'cloud with lightning and rain': '\U000026C8',
    'sun behind small cloud': '\U0001F324',
    'sun behind large cloud': '\U0000F325',
    'sun behind rain cloud ': '\U0001F326',
    'cloud with rain': '\U0001F327',
    'cloud with snow': '\U0001F328',
    'cloud with lightning': '\U0001F329',
    'tornado': '\U0001F32A',
    'fog': '\U0001F32B',
    'wind face': '\U0001F32C'
}

# yr weather symbols numbers
# https://github.com/nrkno/yr-weather-symbols
weather_symbols_yr_numbers = {
    'sun':  ['01'],
    'cloud': ['04'],
    'sun behind cloud': ['03'],
    'cloud with lightning and rain': ['06', '24', '25', '26', '20', '27'],
    'sun behind small cloud': ['02'],
    'sun behind large cloud': [],
    'sun behind rain cloud ': ['40', '05', '41', '42', '07', '43'],
    'cloud with rain': ['46', '09', '10'],
    'cloud with snow': ['44', '08', '45', '28', '21', '29', '47', '12', '48', '49', '13', '50'],
    'cloud with lightning': ['30', '22', '11', '31', '23', '32', '33', '14', '34'],
    'fog': ['15']
}

# yr weather symbols names
# https://github.com/nrkno/yr-weather-symbols
weather_symbols_yr_names = {
    'sun':  ['clearsky_day',
             'clearsky_night'],
    'cloud': ['cloudy'],
    'sun behind cloud': ['partlycloudy_day',
                         'partlycloudy_night'],
    'cloud with lightning and rain': ['lightrainshowersandthunder_day',
                                      'lightrainshowersandthunder_night',
                                      'rainshowersandthunder_day',
                                      'rainshowersandthunder_night',
                                      'heavyrainshowersandthunder_day',
                                      'heavyrainshowersandthunder_night'],
    'sun behind small cloud': ['fair_day',
                               'fair_night'],
    'sun behind large cloud': [],
    'sun behind rain cloud ': ['lightrainshowers_day',
                               'lightrainshowers_night',
                               'rainshowers_day',
                               'rainshowers_night',
                               'heavyrainshowers_day',
                               'heavyrainshowers_night'],
    'cloud with rain': ['rain',
                        'heavyrain'],
    'cloud with snow': ['lightsnowshowers',
                        'snowshowers',
                        'heavysnowshowers',
                        'lightsnowshowersandthunder',
                        'snowshowersandthunder',
                        'heavysnowshowersandthunder',
                        'lightsleet',
                        'sleet',
                        'heavysleet',
                        'lightsnow',
                        'snow',
                        'heavysnow'],
    'cloud with lightning': ['lightrainandthunder',
                             'rainandthunder',
                             'heavyrainandthunder',
                             'lightsleetandthunder',
                             'sleetandthunder',
                             'heavysleetandthunder',
                             'lightsnowandthunder',
                             'snowandthunder',
                             'heavysnowandthunder'],
    'fog': ['fog']
}


In [144]:
print('Precipitation amount: ' + str(yr_precipitation_amount))
print('Precipitation probability: ' + str(yr_precipitation_probability) + ' %')

# get unicode of weather symbol
def get_uni_code(yr_symbol_code):
    search_name = yr_symbol_code
    for uni_name, yr_name_list in weather_symbols_yr_names.items():
        for yr_name in range(len(yr_name_list)):
            if yr_name_list[yr_name] == search_name:
                uni_code = weather_symbols_uni[uni_name]
                return uni_code

print(get_uni_code(yr_symbol_code))



Precipitation amount: 0.0
Precipitation probability: 0.2 %
⛅


In [145]:
# get current time
def get_time():
    time_now = datetime.datetime.now(pytz.timezone("Europe/Berlin"))
    time_now = time_now.strftime("%H:%M")
    return time_now

# get weather data from yr API
def get_weather_data():
        """
        Returns the weather data as a dictionary.
        """

        # send request to YR API
        response = requests.get(api_url, headers=headers)

        # convert response to JSON
        response_json = json.loads(response.text)
        weather_data_json = json.dumps(response_json, indent=4)

        return weather_data_json

# get weather forecast for next hour
def get_forecast(time, weather_data_json):
    """
    Returns the weather forecast for the next hour.
    """

    # get time
    time_now = time

    # get weather data
    weather_data = weather_data_json['properties']['timeseries']
    time_forecast = weather_data_json['properties']['timeseries'][0]['time']

    # get forecast for next hour
    for i in range(len(weather_data)):
        if weather_data[i]['validTime'] == time_now:
            forecast = weather_data[i]['data']['next_6_hours']
            break

    return forecast
    

print(get_time())

15:27
